-sandbox
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

## Boston Housing

Let's build a Keras model to predict the house price on the [Boston Housing dataset](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html).

Description of the dataset from Francois Challot's book: 

>>> You’ll attempt to predict the median price of homes in a given Boston suburb in the mid-1970s, given data points about the suburb at the time, such as the crime rate, the local property tax rate, and so on. 

>>> You have 404 training samples and 102 test samples, each with 13 numerical features, such as per capita crime rate, average number of rooms per dwelling, accessibility to highways, and so on. And each feature in the input data (for example, the crime rate) has a different scale. For instance, some values are proportions, which take values between 0 and 1.The targets are the median values of owner-occupied homes, in thousands of dollars.

>>> The prices are typically between $10,000 and $50,000. If that sounds cheap, rememberthat this was the mid-1970s, and these prices aren’t adjusted for inflation.

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
import numpy as np
np.random.seed(0)

boston_housing = load_boston()

# split 80/20 train-test
X_train, X_test, y_train, y_test = train_test_split(boston_housing.data,
                                                        boston_housing.target,
                                                        test_size=0.2,
                                                        random_state=1)

print(boston_housing.DESCR)

Boston House Prices dataset
===========================

Notes
------
Data Set Characteristics: 

 :Number of Instances: 506 

 :Number of Attributes: 13 numeric/categorical predictive
 
 :Median Value (attribute 14) is usually the target

 :Attribute Information (in order):
 - CRIM per capita crime rate by town
 - ZN proportion of residential land zoned for lots over 25,000 sq.ft.
 - INDUS proportion of non-retail business acres per town
 - CHAS Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
 - NOX nitric oxides concentration (parts per 10 million)
 - RM average number of rooms per dwelling
 - AGE proportion of owner-occupied units built prior to 1940
 - DIS weighted distances to five Boston employment centres
 - RAD index of accessibility to radial highways
 - TAX full-value property-tax rate per $10,000
 - PTRATIO pupil-teacher ratio by town
 - B 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
 - LSTAT % lower status of the population
 - MEDV Median value of owner-occupied homes in $1000's

 :Missing Attribute Values: None

 :Creator: Harrison, D. and Rubinfeld, D.L.

This is a copy of UCI ML housing dataset.
http://archive.ics.uci.edu/ml/datasets/Housing


This dataset was taken from the StatLib library which is maintained at Carnegie Mellon University.

The Boston house-price data of Harrison, D. and Rubinfeld, D.L. 'Hedonic
prices and the demand for clean air', J. Environ. Economics & Management,
vol.5, 81-102, 1978. Used in Belsley, Kuh & Welsch, 'Regression diagnostics
...', Wiley, 1980. N.B. Various transformations are used in the table on
pages 244-261 of the latter.

The Boston house-price data has been used in many machine learning papers that address regression
problems. 
 
**References**

 - Belsley, Kuh & Welsch, 'Regression diagnostics: Identifying Influential Data and Sources of Collinearity', Wiley, 1980. 244-261.
 - Quinlan,R. (1993). Combining Instance-Based and Model-Based Learning. In Proceedings on the Tenth International Conference of Machine Learning, 236-243, University of Massachusetts, Amherst. Morgan Kaufmann.
 - many more! (see http://archive.ics.uci.edu/ml/datasets/Housing)

Let's take a look at the distribution of our features.

In [5]:
import pandas as pd

xTrainDF = pd.DataFrame(X_train, columns=boston_housing.feature_names)

print(xTrainDF.describe())

CRIM ZN INDUS CHAS NOX RM \
count 404.000000 404.000000 404.000000 404.000000 404.000000 404.000000 
mean 3.672702 11.527228 11.077500 0.079208 0.553026 6.268792 
std 9.141351 23.288284 6.848412 0.270398 0.116895 0.689229 
min 0.006320 0.000000 0.460000 0.000000 0.385000 3.561000 
25% 0.082598 0.000000 5.190000 0.000000 0.448000 5.876750 
50% 0.234405 0.000000 9.125000 0.000000 0.535000 6.179000 
75% 3.543428 20.000000 18.100000 0.000000 0.624000 6.626500 
max 88.976200 100.000000 27.740000 1.000000 0.871000 8.780000 

 AGE DIS RAD TAX PTRATIO B \
count 404.000000 404.000000 404.000000 404.000000 404.000000 404.000000 
mean 67.935644 3.826111 9.470297 403.257426 18.438614 357.153688 
std 28.563186 2.120999 8.680237 169.030480 2.169469 91.541647 
min 2.900000 1.129600 1.000000 187.000000 12.600000 0.320000 
25% 43.250000 2.105350 4.000000 277.000000 17.225000 376.092500 
50% 76.800000 3.298600 5.000000 329.000000 19.000000 391.575000 
75% 93.825000 5.141475 24.000000 666.000000 20.200000 396.157500 
max 100.000000 12.126500 24.000000 711.000000 22.000000 396.900000 

 LSTAT 
count 404.000000 
mean 12.778540 
std 7.216403 
min 1.730000 
25% 7.092500 
50% 11.465000 
75% 17.102500 
max 37.970000

## Keras Neural Network Life Cycle Model

![Life Cycle](https://brookewenig.github.io/img/DL/Life-Cycle-for-Neural-Network-Models-in-Keras.png)

# 1. Define a Network

We need to specify our [dense layers](https://keras.io/layers/core/#dense).

The first layer should have 50 units, the second layer, 20 units, and the last layer 1 unit. For all of the layers, make the activation function `relu` except for the last layer, as that activation function should be `linear`.

In [8]:
X_train.shape

Out[ 4 ]: (404, 13)

In [9]:
def build_model():
  return Sequential([Dense(100, input_dim=13, activation='relu'),
                    Dense(40, activation='relu'),
                    Dense(20, activation='relu'),
                    Dense(1, activation='linear')]) # Keep the last layer as linear because this is a regression problem

In [10]:
# TODO
import tensorflow as tf
tf.set_random_seed(42) # For reproducibility
from keras.models import Sequential
from keras.layers import Dense

model = build_model()

## Question

If you did the previous cell correctly, you should see there are 700 parameters for the first layer. Why are there 700?

**HINT**: Add in `use_bias=False` in the Dense layer, and you should see a difference in the number of parameters (don't forget to set this back to `True` before moving on)

In [12]:
# TODO
model.summary()

_________________________________________________________________
Layer (type) Output Shape Param # 
=================================================================
dense_37 (Dense) (None, 100) 1400 
_________________________________________________________________
dense_38 (Dense) (None, 40) 4040 
_________________________________________________________________
dense_39 (Dense) (None, 20) 820 
_________________________________________________________________
dense_40 (Dense) (None, 1) 21 
=================================================================
Total params: 6,281
Trainable params: 6,281
Non-trainable params: 0
_________________________________________________________________

# 2. Compile a Network

To [compile](https://keras.io/getting-started/sequential-model-guide/#compilation) the network, we need to specify the loss function, which optimizer to use, and a metric to evaluate how well the model is performing.

Use `mse` as the loss function, `adam` as the optimizer, and `mse` as the evaluation metric.

In [14]:
# TODO
from keras import optimizers
model.compile(optimizer=optimizers.Adam(lr=0.01), loss='mse')

In [15]:
model

Out[ 63 ]: <keras.models.Sequential at 0x7f56504ba7f0>

# 3. Fit a Network

Now we are going to [fit](https://keras.io/getting-started/sequential-model-guide/#training) our model to our training data. Set `epochs` to 100 and `batch_size` to 32, `verbose` to 2.

In [17]:
# TODO
history = model.fit(X_train, y_train, epochs=1000, batch_size=32, verbose=2)

Epoch 1/1000
 - 0s - loss: 1396.9591
Epoch 2/1000
 - 0s - loss: 125.4969
Epoch 3/1000
 - 0s - loss: 80.4848
Epoch 4/1000
 - 0s - loss: 67.6280
Epoch 5/1000
 - 0s - loss: 59.5480
Epoch 6/1000
 - 0s - loss: 59.4292
Epoch 7/1000
 - 0s - loss: 60.7544
Epoch 8/1000
 - 0s - loss: 59.1632
Epoch 9/1000
 - 0s - loss: 56.3985
Epoch 10/1000
 - 0s - loss: 55.1486
Epoch 11/1000
 - 0s - loss: 54.8388
Epoch 12/1000
 - 0s - loss: 49.9636
Epoch 13/1000
 - 0s - loss: 50.6653
Epoch 14/1000
 - 0s - loss: 46.1052
Epoch 15/1000
 - 0s - loss: 45.0869
Epoch 16/1000
 - 0s - loss: 44.2086
Epoch 17/1000
 - 0s - loss: 43.8372
Epoch 18/1000
 - 0s - loss: 39.1731
Epoch 19/1000
 - 0s - loss: 35.9884
Epoch 20/1000
 - 0s - loss: 39.9442
Epoch 21/1000
 - 0s - loss: 42.5139
Epoch 22/1000
 - 0s - loss: 38.0223
Epoch 23/1000
 - 0s - loss: 33.8857
Epoch 24/1000
 - 0s - loss: 29.6643
Epoch 25/1000
 - 0s - loss: 28.7376
Epoch 26/1000
 - 0s - loss: 30.1054
Epoch 27/1000
 - 0s - loss: 37.9000
Epoch 28/1000
 - 0s - loss: 35.3038
Epoch 29/1000
 - 0s - loss: 29.4215
Epoch 30/1000
 - 0s - loss: 33.9563
Epoch 31/1000
 - 0s - loss: 38.6181
Epoch 32/1000
 - 0s - loss: 35.9324
Epoch 33/1000
 - 0s - loss: 30.6021
Epoch 34/1000
 - 0s - loss: 23.7432
Epoch 35/1000
 - 0s - loss: 25.7718
Epoch 36/1000
 - 0s - loss: 22.9125
Epoch 37/1000
 - 0s - loss: 19.9318
Epoch 38/1000
 - 0s - loss: 19.6824
Epoch 39/1000
 - 0s - loss: 24.7022
Epoch 40/1000
 - 0s - loss: 23.4314
Epoch 41/1000
 - 0s - loss: 21.7055
Epoch 42/1000
 - 0s - loss: 24.1147
Epoch 43/1000
 - 0s - loss: 19.8560
Epoch 44/1000
 - 0s - loss: 24.7238
Epoch 45/1000
 - 0s - loss: 30.3160
Epoch 46/1000
 - 0s - loss: 25.5087
Epoch 47/1000
 - 0s - loss: 23.6023
Epoch 48/1000
 - 0s - loss: 26.9612
Epoch 49/1000
 - 0s - loss: 22.3026
Epoch 50/1000
 - 0s - loss: 22.8489
Epoch 51/1000
 - 0s - loss: 20.8441
Epoch 52/1000
 - 0s - loss: 23.2304
Epoch 53/1000
 - 0s - loss: 18.0690
Epoch 54/1000
 - 0s - loss: 19.8824
Epoch 55/1000
 - 0s - loss: 18.8670
Epoch 56/1000
 - 0s - loss: 19.7069
Epoch 57/1000
 - 0s - loss: 18.0118
Epoch 58/1000
 - 0s - loss: 24.5286
Epoch 59/1000
 - 0s - loss: 23.3649
Epoch 60/1000
 - 0s - loss: 21.1614
Epoch 61/1000
 - 0s - loss: 20.1197
Epoch 62/1000
 - 0s - loss: 22.7247
Epoch 63/1000
 - 0s - loss: 21.6074
Epoch 64/1000
 - 0s - loss: 17.8376
Epoch 65/1000
 - 0s - loss: 17.9582
Epoch 66/1000
 - 0s - loss: 18.8286
Epoch 67/1000
 - 0s - loss: 23.6223
Epoch 68/1000
 - 0s - loss: 25.2500
Epoch 69/1000
 - 0s - loss: 22.8479
Epoch 70/1000
 - 0s - loss: 21.8901
Epoch 71/1000
 - 0s - loss: 18.2761
Epoch 72/1000
 - 0s - loss: 17.9217
Epoch 73/1000
 - 0s - loss: 20.8047
Epoch 74/1000
 - 0s - loss: 25.7565
Epoch 75/1000
 - 0s - loss: 24.4070
Epoch 76/1000
 - 0s - loss: 20.5569
Epoch 77/1000
 - 0s - loss: 19.7403
Epoch 78/1000
 - 0s - loss: 21.9621
Epoch 79/1000
 - 0s - loss: 24.0597
Epoch 80/1000
 - 0s - loss: 21.8001
Epoch 81/1000
 - 0s - loss: 28.4385
Epoch 82/1000
 - 0s - loss: 18.8442
Epoch 83/1000
 - 0s - loss: 17.8368
Epoch 84/1000
 - 0s - loss: 18.6428
Epoch 85/1000
 - 0s - loss: 20.2238
Epoch 86/1000
 - 0s - loss: 19.1459
Epoch 87/1000
 - 0s - loss: 21.2189
Epoch 88/1000
 - 0s - loss: 24.0188
Epoch 89/1000
 - 0s - loss: 18.7229
Epoch 90/1000
 - 0s - loss: 17.4205
Epoch 91/1000
 - 0s - loss: 17.3363
Epoch 92/1000
 - 0s - loss: 17.0227
Epoch 93/1000
 - 0s - loss: 17.6286
Epoch 94/1000
 - 0s - loss: 18.6134
Epoch 95/1000
 - 0s - loss: 19.5880
Epoch 96/1000
 - 0s - loss: 20.5536
Epoch 97/1000
 - 0s - loss: 28.1108
Epoch 98/1000
 - 0s - loss: 23.9930
Epoch 99/1000
 - 0s - loss: 19.1089
Epoch 100/1000
 - 0s - loss: 18.8100
Epoch 101/1000
 - 0s - loss: 20.4912
Epoch 102/1000
 - 0s - loss: 19.4227
Epoch 103/1000
 - 0s - loss: 16.6857
Epoch 104/1000
 - 0s - loss: 19.9435
Epoch 105/1000
 - 0s - loss: 17.4825
Epoch 106/1000
 - 0s - loss: 24.9368
Epoch 107/1000
 - 0s - loss: 18.9862
Epoch 108/1000
 - 0s - loss: 18.6092
Epoch 109/1000
 - 0s - loss: 18.9503
Epoch 110/1000
 - 0s - loss: 16.9755
Epoch 111/1000
 - 0s - loss: 17.732

Visualize model loss

In [19]:
import matplotlib.pyplot as plt

def viewModelLoss():
  plt.clf()
  plt.plot(history.history['loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  display(plt.show())
viewModelLoss()

# 4. Evaluate Network

In [21]:
# TODO
model.evaluate(X_test, y_test)

32/102 [========>.....................] - ETA: 0s
102/102 [==============================] - 0s 1ms/step
 Out[ 78 ]: 14.938928379732019

# 5. Make Predictions

In [23]:
# TODO
pred1 = model.predict(X_test)
pred1-y_test

Out[ 37 ]: 
array([[ 6.41259842, 10.71259842, 18.01259842, ..., 12.91259842,
 24.41259842, 19.21259842],
 [ -0.48397751, 3.81602249, 11.11602249, ..., 6.01602249,
 17.51602249, 12.31602249],
 [ -9.26945992, -4.96945992, 2.33054008, ..., -2.76945992,
 8.73054008, 3.53054008],
 ...,
 [ -7.63624306, -3.33624306, 3.96375694, ..., -1.13624306,
 10.36375694, 5.16375694],
 [-16.81186028, -12.51186028, -5.21186028, ..., -10.31186028,
 1.18813972, -4.01186028],
 [-13.38111229, -9.08111229, -1.78111229, ..., -6.88111229,
 4.61888771, -0.58111229]])

Wahoo! You successfully just built your first neural network in Keras!

## BONUS:
Try around with changing some hyperparameters. See what happens if you increase the number of layers, or change the optimizer, etc. What about normalizing the data??

If you have time, how about building a baseline model to compare against?

-sandbox
&copy; 2018 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>